In [3]:
import numpy as np
import scipy.optimize as opti
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')
from scipy.interpolate import griddata
from scipy.interpolate import rbf
from scipy.interpolate import LinearNDInterpolator
import cPickle as pickle

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import sys
sys.path.append('/Users/vonderlinden2/rsx_analysis/writing_to_vtk/source')
import structured_3d_vtk as struc_3d
reload(struc_3d)

<module 'structured_3d_vtk' from '/Users/vonderlinden2/rsx_analysis/writing_to_vtk/source/structured_3d_vtk.pyc'>

In [7]:
spatial_increment = 0.001
x_min, x_max = -0.028, 0.025
y_min, y_max = -0.043, 0.039
z_min, z_max = 0.249, 0.249

bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

x_coord = np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment))
y_coord = np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment))

mesh = np.meshgrid(x_coord, y_coord, indexing='ij')
grid_points = np.dstack(map(np.ravel, mesh))[0]

# Interpolate linearly

In [8]:
file_path = '../output/2016-07-03/B_z0249_'
ending = '.npz'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

for plane in [0.249, 0.302, 0.357, 0.416]:
    bx_measurements[plane] = struc_3d.average_duplicate_points(bx_measurements[plane])
    by_measurements[plane] = struc_3d.average_duplicate_points(by_measurements[plane])
    bz_measurements[plane] = struc_3d.average_duplicate_points(bz_measurements[plane])

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 

interpolated_vectors = []
for time_point in xrange(21):
    print time_point
    points = []
    values = []
    for measurements in direction_measurements:
        (points_direction,
         values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                        time_point,
                                                                        [0.249])
        points.append(points_direction)
        values.append(values_direction)
    
    points[0] = np.delete(points[0], 2, axis=1)
    points[1] = np.delete(points[1], 2, axis=1)
    points[2] = np.delete(points[2], 2, axis=1)
    interpolated_vector = struc_3d.interpolate_vector(grid_points, points, values)
    #interpolated_vector = struc_3d.add_vacuum_field(interpolated_vector)
    
    assert np.sum(np.isnan(interpolated_vector[0])) == 0
    assert np.sum(np.isnan(interpolated_vector[1])) == 0
    assert np.sum(np.isnan(interpolated_vector[2])) == 0
    
    
    sizes = (np.unique(grid_points[:, 0]).size,
             np.unique(grid_points[:, 1]).size)
    
    grid_swapped = np.swapaxes(grid_points, 0, 1)
    grid_x = np.resize(grid_swapped[0], (sizes[0], sizes[1]))
    grid_y = np.resize(grid_swapped[1], (sizes[0], sizes[1]))
    grid = np.asarray([grid_x, grid_y])
    
    interpolated_vector = np.asarray(interpolated_vector)
    vector_swapped = np.swapaxes(interpolated_vector, 0, 1)
    vector_x = np.resize(vector_swapped[:, 0], (sizes[0], sizes[1]))
    vector_y = np.resize(vector_swapped[:, 1], (sizes[0], sizes[1]))
    vector = np.asarray([vector_x, vector_y])
    
    interpolated_vectors.append(vector)
    
    np.savez(file_path + str(time_point).zfill(4) + ending,
             b=vector, 
             grid=grid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [14]:
file_path = '../output/2016-07-03/B_z0249_'
ending = '.p'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

for plane in [0.249, 0.302, 0.357, 0.416]:
    bx_measurements[plane] = struc_3d.average_duplicate_points(bx_measurements[plane])
    by_measurements[plane] = struc_3d.average_duplicate_points(by_measurements[plane])
    bz_measurements[plane] = struc_3d.average_duplicate_points(bz_measurements[plane])

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 

interpolators = []
for time_point in xrange(21):
    print time_point
    points = []
    values = []
    for measurements in direction_measurements:
        (points_direction,
         values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                        time_point,
                                                                        [0.249])
        points.append(points_direction)
        values.append(values_direction)
    
    points[0] = np.delete(points[0], 2, axis=1)
    points[1] = np.delete(points[1], 2, axis=1)
    points[2] = np.delete(points[2], 2, axis=1)
    interpolator_x = LinearNDInterpolator(points[0], values[0])
    interpolator_y = LinearNDInterpolator(points[1], values[1])
    interpolator_z = LinearNDInterpolator(points[2], values[2])
    pickle.dump(interpolator_x, open(file_path + 'x_' + str(time_point).zfill(4) + ending, 'wb'))
    pickle.dump(interpolator_y, open(file_path + 'y_' + str(time_point).zfill(4) + ending, 'wb'))
    pickle.dump(interpolator_z, open(file_path + 'z_' + str(time_point).zfill(4) + ending, 'wb'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


# Examine grid ordering

In [14]:
file_path = '../output/2016-07-03/B_z0249_'
ending = '.npz'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

for plane in [0.249, 0.302, 0.357, 0.416]:
    bx_measurements[plane] = struc_3d.average_duplicate_points(bx_measurements[plane])
    by_measurements[plane] = struc_3d.average_duplicate_points(by_measurements[plane])
    bz_measurements[plane] = struc_3d.average_duplicate_points(bz_measurements[plane])

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 


time_point=0
points = []
values = []
measurements = bx_measurements

(points_direction,
 values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                time_point,
                                                                [0.249])
points = points_direction
values = values_direction
    
points = np.delete(points, 2, axis=1)
   
interpolated_scalar = struc_3d.interpolate_scalar(grid_points, points, values)
#interpolated_vector = struc_3d.add_vacuum_field(interpolated_vector)
    
assert np.sum(np.isnan(interpolated_scalar)) == 0
    
sizes = (np.unique(grid_points[:, 0]).size,
         np.unique(grid_points[:, 1]).size)
    
grid_swapped = np.swapaxes(grid_points, 0, 1)
grid_x = np.resize(grid_swapped[0], (sizes[0], sizes[1]))
grid_y = np.resize(grid_swapped[1], (sizes[0], sizes[1]))
grid = np.asarray([grid_x, grid_y])
    
interpolated_scalar = np.asarray(interpolated_scalar)
scalar = np.resize(interpolated_scalar, (sizes[0], sizes[1]))


In [15]:
scalar

array([[ -1.60520156e-04,  -1.62398882e-04,  -1.73493664e-04, ...,
          1.92798563e-04,   1.92182204e-04,   1.91565846e-04],
       [ -1.44041504e-04,  -1.45920231e-04,  -1.50443651e-04, ...,
          2.95728433e-04,   2.68979060e-04,   2.68044026e-04],
       [ -2.25491089e-04,  -2.27369815e-04,  -2.44343123e-04, ...,
          2.82471044e-04,   2.55721671e-04,   2.37861818e-04],
       ..., 
       [ -6.80633642e-06,  -5.52919785e-05,  -1.28570143e-04, ...,
         -1.15569618e-04,  -2.18158869e-04,  -2.19888576e-04],
       [  2.31864552e-05,  -3.76954042e-05,  -1.10973569e-04, ...,
         -7.87077349e-05,  -1.57095065e-04,  -1.58824772e-04],
       [ -8.49811979e-05,  -7.86033894e-05,  -7.22255809e-05, ...,
         -4.18458521e-05,  -9.67283577e-05,  -9.77609681e-05]])

In [16]:
interpolated_scalar

array([ -1.60520156e-04,  -1.62398882e-04,  -1.73493664e-04, ...,
        -4.18458521e-05,  -9.67283577e-05,  -9.77609681e-05])

In [17]:
grid_points

array([[-0.028     , -0.043     ],
       [-0.028     , -0.04198765],
       [-0.028     , -0.04097531],
       ..., 
       [ 0.025     ,  0.03697531],
       [ 0.025     ,  0.03798765],
       [ 0.025     ,  0.039     ]])

In [33]:
grid[1][0][2]

-0.040975308641975308

In [30]:
scalar[0][2]

-0.00017349366393440481

In [40]:
scalar[0][1] - scalar[0][0] 

-1.8787261375692609e-06

In [39]:
scalar[0][1]

-0.00016239888172420871

In [44]:
np.diff(scalar, axis=1)

array([[ -1.87872614e-06,  -1.10947822e-05,  -8.55028609e-05, ...,
         -1.70610395e-05,  -6.16358521e-07,  -6.16358521e-07],
       [ -1.87872614e-06,  -4.52342036e-06,  -1.10306614e-04, ...,
         -2.67493733e-05,  -2.67493733e-05,  -9.35034135e-07],
       [ -1.87872614e-06,  -1.69733081e-05,  -8.39879825e-05, ...,
         -2.67247240e-05,  -2.67493733e-05,  -1.78598530e-05],
       ..., 
       [ -4.84856420e-05,  -7.32781649e-05,  -6.88929429e-05, ...,
         -5.48825056e-05,  -1.02589251e-04,  -1.72970748e-06],
       [ -6.08818594e-05,  -7.32781649e-05,  -6.88929429e-05, ...,
         -5.48825056e-05,  -7.83873299e-05,  -1.72970748e-06],
       [  6.37780852e-06,   6.37780852e-06,   6.37780852e-06, ...,
         -5.48825056e-05,  -5.48825056e-05,  -1.03261038e-06]])

In [43]:
np.diff(scalar, axis=0)

array([[  1.64786511e-05,   1.64786511e-05,   2.30500130e-05, ...,
          1.02929870e-04,   7.67968557e-05,   7.64781801e-05],
       [ -8.14495845e-05,  -8.14495845e-05,  -9.38994723e-05, ...,
         -1.32573892e-05,  -1.32573892e-05,  -3.01822081e-05],
       [ -8.14495845e-05,  -8.89968659e-05,  -9.60050400e-05, ...,
         -1.32738217e-05,  -1.32573892e-05,  -1.32573892e-05],
       ..., 
       [  2.99927916e-05,   2.55036554e-05,   1.75965742e-05, ...,
          5.16021898e-05,   6.10638042e-05,   6.10638042e-05],
       [  2.99927916e-05,   1.75965742e-05,   1.75965742e-05, ...,
          3.68618828e-05,   6.10638042e-05,   6.10638042e-05],
       [ -1.08167653e-04,  -4.09079852e-05,   3.87479882e-05, ...,
          3.68618828e-05,   6.03667071e-05,   6.10638042e-05]])

In [47]:
np.gradient(scalar)[0]

array([[  1.64786511e-05,   1.64786511e-05,   2.30500130e-05, ...,
          1.02929870e-04,   7.67968557e-05,   7.64781801e-05],
       [ -3.24854667e-05,  -3.24854667e-05,  -3.54247297e-05, ...,
          4.48362406e-05,   3.17697332e-05,   2.31479860e-05],
       [ -8.14495845e-05,  -8.52232252e-05,  -9.49522561e-05, ...,
         -1.32656055e-05,  -1.32573892e-05,  -2.17197986e-05],
       ..., 
       [  2.99927916e-05,   2.15501148e-05,   1.75965742e-05, ...,
          4.42320363e-05,   6.10638042e-05,   6.10638042e-05],
       [ -3.90874308e-05,  -1.16557055e-05,   2.81722812e-05, ...,
          3.68618828e-05,   6.07152556e-05,   6.10638042e-05],
       [ -1.08167653e-04,  -4.09079852e-05,   3.87479882e-05, ...,
          3.68618828e-05,   6.03667071e-05,   6.10638042e-05]])

In [49]:
np.gradient(scalar)[1]

array([[ -1.87872614e-06,  -6.48675417e-06,  -4.82988216e-05, ...,
         -8.83869902e-06,  -6.16358521e-07,  -6.16358521e-07],
       [ -1.87872614e-06,  -3.20107325e-06,  -5.74150173e-05, ...,
         -2.67493733e-05,  -1.38422037e-05,  -9.35034135e-07],
       [ -1.87872614e-06,  -9.42601712e-06,  -5.04806453e-05, ...,
         -2.67370487e-05,  -2.23046132e-05,  -1.78598530e-05],
       ..., 
       [ -4.84856420e-05,  -6.08819035e-05,  -7.10855539e-05, ...,
         -7.87358784e-05,  -5.21594793e-05,  -1.72970748e-06],
       [ -6.08818594e-05,  -6.70800122e-05,  -7.10855539e-05, ...,
         -6.66349178e-05,  -4.00585187e-05,  -1.72970748e-06],
       [  6.37780852e-06,   6.37780852e-06,   6.37780852e-06, ...,
         -5.48825056e-05,  -2.79575580e-05,  -1.03261038e-06]])

In [45]:
scalar[1, 0] - scalar[0,0]

1.6478651129742952e-05

# Interpolate cubic with C-T method

In [81]:
file_path = '../output/2016-07-03/B_z0249_cubic'
ending = '.npz'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 


interpolated_vectors = []
for time_point in xrange(21):
    print time_point
    points = []
    values = []
    for measurements in direction_measurements:
        (points_direction,
         values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                        time_point,
                                                                        [0.249])
        points.append(points_direction)
        values.append(values_direction)
    
    points[0] = np.delete(points[0], 2, axis=1)
    points[1] = np.delete(points[1], 2, axis=1)
    points[2] = np.delete(points[2], 2, axis=1)
    interpolated_vector = struc_3d.interpolate_vector(grid_points, points, values, method='cubic')
    #interpolated_vector = struc_3d.add_vacuum_field(interpolated_vector)
    
    assert np.sum(np.isnan(interpolated_vector[0])) == 0
    assert np.sum(np.isnan(interpolated_vector[1])) == 0
    assert np.sum(np.isnan(interpolated_vector[2])) == 0
    
    
    sizes = (np.unique(grid_points[:, 0]).size,
             np.unique(grid_points[:, 1]).size)
    
    grid_swapped = np.swapaxes(grid_points, 0, 1)
    grid_x = np.resize(grid_swapped[0], (sizes[0], sizes[1]))
    grid_y = np.resize(grid_swapped[1], (sizes[0], sizes[1]))
    grid = np.asarray([grid_x, grid_y])
    
    interpolated_vector = np.asarray(interpolated_vector)
    vector_swapped = np.swapaxes(interpolated_vector, 0, 1)
    vector_x = np.resize(vector_swapped[:, 0], (sizes[0], sizes[1]))
    vector_y = np.resize(vector_swapped[:, 1], (sizes[0], sizes[1]))
    vector = np.asarray([vector_x, vector_y])
    
    interpolated_vectors.append(vector)
    
    np.savez(file_path + str(time_point).zfill(4) + ending,
             b=vector, 
             grid=grid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


# Use radial basis functions

In [98]:
from scipy.interpolate import Rbf

In [209]:
def average_duplicate_points(data_dict):
    data_dict['x_out'] = data_dict['x_out'].astype('float64')
    data_dict['y_out'] = data_dict['y_out'].astype('float64')
    data_dict['a_out'] = data_dict['a_out'].astype('float64')
    time_points = data_dict['a_out'].shape[0]
    data = {}
    for idx in xrange(data_dict['x_out'].size):
        location = (data_dict['x_out'][idx], data_dict['y_out'][idx])
        if location in data.keys():
            data[location] = np.column_stack((data[location], data_dict['a_out'][:, idx]))
        else:
            data[location] = data_dict['a_out'][:, idx] 

    unique_data_dict = {'x_out': [],
                        'y_out': [],
                        'a_out': [],
                        'std': []}
    for location in data.keys():
        if data[location][0].size > 1:
            unique_data_dict['std'].append(data[location].std(axis=1, ddof=1))
            unique_data_dict['a_out'].append(data[location].mean(axis=1))  
        else:
            unique_data_dict['std'].append(np.zeros(time_points))
            unique_data_dict['a_out'].append(data[location])  
        unique_data_dict['x_out'].append(location[0])
        unique_data_dict['y_out'].append(location[1])

    unique_data_dict['x_out'] = np.asarray(unique_data_dict['x_out'])
    unique_data_dict['y_out'] = np.asarray(unique_data_dict['y_out'])
    test = np.asarray(unique_data_dict['a_out'])
    unique_data_dict['a_out'] = np.hsplit(np.asarray(unique_data_dict['a_out']), time_points)
    unique_data_dict['delays'] = data_dict['delays']
    return unique_data_dict

In [212]:
bx_measurements.keys()

[0.249, 0.302, 0.357, 0.416]

In [214]:
file_path = '../output/2016-07-03/B_z0249_rbf_thin_plate_'
ending = '.npz'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

bx_measurements[0.249] = average_duplicate_points(bx_measurements[0.249])
by_measurements[0.249] = average_duplicate_points(by_measurements[0.249])
bz_measurements[0.249] = average_duplicate_points(bz_measurements[0.249])

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 


interpolated_vectors = []
for time_point in xrange(21):
    print time_point
    points = []
    values = []
    for measurements in direction_measurements:
        (points_direction,
         values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                        time_point,
                                                                        [0.249])
        points.append(points_direction)
        values.append(values_direction)
    
    points[0] = np.delete(points[0], 2, axis=1)
    points[1] = np.delete(points[1], 2, axis=1)
    points[2] = np.delete(points[2], 2, axis=1)
    
    rbf_func_x = Rbf(points[0][:, 0], points[0][:, 1], values[0], function='thin_plate')
    rbf_func_y = Rbf(points[0][:, 0], points[0][:, 1], values[0], function='thin_plate')
    rbf_func_z = Rbf(points[0][:, 0], points[0][:, 1], values[0], function='thin_plate')
    
    interpolated_vector[0] = rbf_func_x(grid_points[:, 0], grid_points[:, 1])
    interpolated_vector[1] = rbf_func_y(grid_points[:, 0], grid_points[:, 1])
    interpolated_vector[2] = rbf_func_z(grid_points[:, 0], grid_points[:, 1])
    
    assert np.sum(np.isnan(interpolated_vector[0])) == 0
    assert np.sum(np.isnan(interpolated_vector[1])) == 0
    assert np.sum(np.isnan(interpolated_vector[2])) == 0
    
    
    sizes = (np.unique(grid_points[:, 0]).size,
             np.unique(grid_points[:, 1]).size)
    
    grid_swapped = np.swapaxes(grid_points, 0, 1)
    grid_x = np.resize(grid_swapped[0], (sizes[0], sizes[1]))
    grid_y = np.resize(grid_swapped[1], (sizes[0], sizes[1]))
    grid = np.asarray([grid_x, grid_y])
    
    interpolated_vector = np.asarray(interpolated_vector)
    vector_swapped = np.swapaxes(interpolated_vector, 0, 1)
    vector_x = np.resize(vector_swapped[:, 0], (sizes[0], sizes[1]))
    vector_y = np.resize(vector_swapped[:, 1], (sizes[0], sizes[1]))
    vector = np.asarray([vector_x, vector_y])
    
    interpolated_vectors.append(vector)
    
    np.savez(file_path + str(time_point).zfill(4) + ending,
             b=vector, 
             grid=grid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [216]:
file_path = '../output/2016-07-03/B_z0249_rbf_cubic_'
ending = '.npz'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

bx_measurements[0.249] = average_duplicate_points(bx_measurements[0.249])
by_measurements[0.249] = average_duplicate_points(by_measurements[0.249])
bz_measurements[0.249] = average_duplicate_points(bz_measurements[0.249])

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 


interpolated_vectors = []
for time_point in xrange(21):
    print time_point
    points = []
    values = []
    for measurements in direction_measurements:
        (points_direction,
         values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                        time_point,
                                                                        [0.249])
        points.append(points_direction)
        values.append(values_direction)
    
    points[0] = np.delete(points[0], 2, axis=1)
    points[1] = np.delete(points[1], 2, axis=1)
    points[2] = np.delete(points[2], 2, axis=1)
    
    rbf_func_x = Rbf(points[0][:, 0], points[0][:, 1], values[0], function='cubic')
    rbf_func_y = Rbf(points[0][:, 0], points[0][:, 1], values[0], function='cubic')
    rbf_func_z = Rbf(points[0][:, 0], points[0][:, 1], values[0], function='cubic')
    
    interpolated_vector[0] = rbf_func_x(grid_points[:, 0], grid_points[:, 1])
    interpolated_vector[1] = rbf_func_y(grid_points[:, 0], grid_points[:, 1])
    interpolated_vector[2] = rbf_func_z(grid_points[:, 0], grid_points[:, 1])
    
    assert np.sum(np.isnan(interpolated_vector[0])) == 0
    assert np.sum(np.isnan(interpolated_vector[1])) == 0
    assert np.sum(np.isnan(interpolated_vector[2])) == 0
    
    
    sizes = (np.unique(grid_points[:, 0]).size,
             np.unique(grid_points[:, 1]).size)
    
    grid_swapped = np.swapaxes(grid_points, 0, 1)
    grid_x = np.resize(grid_swapped[0], (sizes[0], sizes[1]))
    grid_y = np.resize(grid_swapped[1], (sizes[0], sizes[1]))
    grid = np.asarray([grid_x, grid_y])
    
    interpolated_vector = np.asarray(interpolated_vector)
    vector_swapped = np.swapaxes(interpolated_vector, 0, 1)
    vector_x = np.resize(vector_swapped[:, 0], (sizes[0], sizes[1]))
    vector_y = np.resize(vector_swapped[:, 1], (sizes[0], sizes[1]))
    vector = np.asarray([vector_x, vector_y])
    
    interpolated_vectors.append(vector)
    
    np.savez(file_path + str(time_point).zfill(4) + ending,
             b=vector, 
             grid=grid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
